In [1]:
import numpy as np
import pandas as pd
from const import embedding_path
embeddings = np.load(embedding_path, allow_pickle=True)
df = pd.DataFrame(embeddings, columns=['employee', 'embedding'])
print(df.shape)
print(df)

(51, 2)
    employee                                          embedding
0        huy  [1.3946319818496704, -0.2541764974594116, -1.1...
1        huy  [-0.7068958282470703, -0.05882064998149872, -1...
2        huy  [0.45191165804862976, 0.28204864263534546, -2....
3        huy  [0.3449893891811371, -0.6703435182571411, -3.0...
4        huy  [1.2440565824508667, -0.23258879780769348, -3....
5        huy  [0.8567330837249756, 0.5315626859664917, -3.41...
6        huy  [0.6111795902252197, 0.04968537390232086, -2.0...
7        huy  [0.8580203056335449, -0.2516350746154785, -3.0...
8        huy  [0.945615291595459, -0.25987350940704346, -2.8...
9        huy  [1.0543687343597412, -0.7749121189117432, -1.6...
10       huy  [1.2935665845870972, -0.20559996366500854, -2....
11       huy  [0.8920835256576538, -0.46103379130363464, -1....
12       huy  [0.33227694034576416, 0.6263231039047241, -3.2...
13       huy  [1.2165075540542603, -0.35650166869163513, -2....
14       huy  [1.145741581916809

In [10]:
import os
from const import embedding_path
from view.utils.data import add_img2db, get_face_pixels, detect_face
import numpy as np
from view.utils.lite_predict import predict_tfmodel


dataset_path = "/home/iwin/Desktop/df_non_tf/static/snap_shots"
for root, dirs, files in os.walk(dataset_path,  topdown=False):
    for name in files:
        try:
            label = root[root.rfind('/')+1:]
            img = detect_face(os.path.join(root, name))
            if os.path.isfile(embedding_path):
                embeddings = np.load(embedding_path, allow_pickle=True)
            else:
                embeddings = np.zeros(shape=(0, 2))
            img_pixels = get_face_pixels(img)
            embedding = predict_tfmodel(img_pixels)[0].tolist()
            embedding = np.array(embedding)
            new_embeddings = np.concatenate(
                [embeddings, np.array([label, embedding]).reshape(1, 2)], axis=0)
            np.save(embedding_path, new_embeddings)
            print(name)
        except Exception as err:
            print(err)
            continue
print('------------')

<ipython-input-10-366853ef5d46>:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  [embeddings, np.array([label, embedding]).reshape(1, 2)], axis=0)
1621406887.jpg
1621406938.jpg
1621406917.jpg
Face could not be detected. Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
1621406903.jpg
1621406941.jpg
Face could not be detected. Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
1621406924.jpg
1621406885.jpg
1621406919.jpg
1621406939.jpg
Face could not be detected. Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
1621406884.jpg
1621406925.jpg
1621406886.jpg
1621406920.jpg
Face could not be detected. Please confirm that 

In [1]:
import cv2
from const import input_shape_size, w_min
from utils import load_database
from custom_deepface.deepface.commons import functions

df, face_cascade = load_database()
list_img_path = '/home/iwin/Desktop/dataset/84/A_84_0.jpg'
img = cv2.imread(list_img_path)
while(img.shape[0] > input_shape_size):
    img = cv2.resize(img, (int(img.shape[1]/2), int(img.shape[0]/2)))
faces = face_cascade.detectMultiScale(img,  1.3, 5)
for (x, y, w, h) in faces:
    if w > w_min:  # discard small detected faces
        # -------------------------------
        # apply deep learning for custom_face
        base_img = img.copy()
        img, region = functions.detect_face(
            img=img, enforce_detection=False)
        # --------------------------

        if img.shape[0] > 0 and img.shape[1] > 0:
            img = functions.align_face(img=img)
        else:
            img = base_img.copy()
cv2.imshow(list_img_path, img)
cv2.waitKey()

In [ ]:
from const import input_shape, threshold
import numpy as np
from view.utils.lite_predict import predict_tfmodel
from custom_deepface.deepface.commons import distance as dst

# post-processing
img = cv2.resize(img, input_shape)
img_pixels = np.array(img, dtype=np.float32)
face_pixels = np.expand_dims(img_pixels, axis=0)
face_pixels /= 255  # normalize input in [0, 1]

# check preprocess_face function handled
if face_pixels.shape[1:3] == input_shape:
    if df.shape[0] > 0:
        img1_representation = predict_tfmodel(face_pixels)[
            0, :]

        def findDistance(row):
            img2_representation = row['embedding']
            distance = dst.findCosineDistance(
                img1_representation, img2_representation)
            return distance
        
        df['distance'] = df.apply(findDistance, axis=1)
        df = df.sort_values(by=["distance"])
        
        list_distance = df.iloc[0:3]['distance'].tolist()
        list_candidates = df.iloc[0:3]['employee'].tolist()
        if list_distance[0] > threshold:
            candidate_label = 'unknown'
        else:
            if (list_candidates.count(list_candidates[0]) >= 2):
                candidate_label = list_candidates[0]
            elif (list_distance[1] < threshold and list_candidates.count(list_candidates[1]) >= 2):
                candidate_label = list_candidates[1]
            else:
                candidate_label = 'unknown'
print(candidate_label)

In [1]:
import cv2
from const import input_shape_size, w_min
from utils import load_database
from custom_deepface.deepface.commons import functions
from const import input_shape, threshold
import numpy as np
from view.utils.lite_predict import predict_tfmodel
from custom_deepface.deepface.commons import distance as dst
import time


prev = time.time()
df, face_cascade = load_database()
list_img_path = '/home/iwin/Desktop/dataset/77/A_77_0.jpg'
img = cv2.imread(list_img_path)
while(img.shape[0] > input_shape_size):
    img = cv2.resize(img, (int(img.shape[1]/2), int(img.shape[0]/2)))
faces = face_cascade.detectMultiScale(img,  1.3, 5)
for (x, y, w, h) in faces:
    if w > w_min:  # discard small detected faces
        # -------------------------------
        # apply deep learning for custom_face
        base_img = img.copy()
        img, region = functions.detect_face(grayscale=True,
            img=img, enforce_detection=False)
        # --------------------------

        if img.shape[0] > 0 and img.shape[1] > 0:
            img = functions.align_face(img=img)
        else:
            img = base_img.copy()
        # cv2.imwrite(list_img_path, img)
        # post-processing
        img = cv2.resize(img, input_shape)
        img_pixels = np.array(img, dtype=np.float32)
        face_pixels = np.expand_dims(img_pixels, axis=0)
        face_pixels /= 255  # normalize input in [0, 1]

        # check preprocess_face function handled
        if face_pixels.shape[1:3] == input_shape:
            if df.shape[0] > 0:
                img1_representation = predict_tfmodel(face_pixels)[
                    0, :]

                def findDistance(row):
                    img2_representation = row['embedding']
                    distance = dst.findCosineDistance(
                        img1_representation, img2_representation)
                    return distance
                
                df['distance'] = df.apply(findDistance, axis=1)
                df = df.sort_values(by=["distance"])
                print(df)
                print(threshold)
                list_distance = df.iloc[0:3]['distance'].tolist()
                list_candidates = df.iloc[0:3]['employee'].tolist()
                if list_distance[0] > threshold:
                    candidate_label = 'unknown'
                else:
                    if (list_candidates.count(list_candidates[0]) >= 2):
                        candidate_label = list_candidates[0]
                    elif (list_distance[1] < threshold and list_candidates.count(list_candidates[1]) >= 2):
                        candidate_label = list_candidates[1]
                    else:
                        candidate_label = 'unknown'
        print(candidate_label)
print(time.time() - prev)
# cv2.imshow("img",img)
cv2.waitKey()

     employee                                          embedding  \
1856       77  [0.6798121929168701, -0.2195548564195633, -1.0...   
1861       77  [0.5230937004089355, -0.15382125973701477, -1....   
1863       77  [0.14271309971809387, -0.16266964375972748, -1...   
1874       77  [0.3807269334793091, -0.31144365668296814, -1....   
1867       77  [0.4730348587036133, -0.15449300408363342, -1....   
...       ...                                                ...   
972        11  [-0.4137284457683563, -0.13159596920013428, -0...   
2712       03  [-1.5341730117797852, -1.1738831996917725, 0.6...   
2942       82  [-0.6154383420944214, -0.3314163386821747, -0....   
2696       03  [-1.6036924123764038, -0.8304492235183716, 0.8...   
925        02  [-1.276563286781311, -0.2367156594991684, 1.41...   

     distance_metric      distance  
1856          cosine  7.836864e-09  
1861          cosine  1.048347e-02  
1863          cosine  4.072545e-02  
1874          cosine  4.263044e-02 

-1